In [12]:
from openpyxl.styles import Font
from copy import copy
from app.main.models import EducationStaff, ExcelStyle
import requests
from config import ApeksAPI, FlaskConfig
from app.main.func import db_filter_req, db_request
from datetime import date
from calendar import monthrange
from openpyxl import load_workbook

def get_lessons(year, month):
    """List of lessons for selected month"""
    first_day = 1
    last_day = monthrange(year, month)[1]

    params = {
        "token": ApeksAPI.TOKEN,
        "table": 'schedule_day_schedule_lessons',
        "filter": "date between '" + date(year, month, first_day).isoformat() + "' and '" + date(year, month, last_day).isoformat() + "'"
    }
    resp = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params)
    return resp.json()['data']


def get_departments():
    """Getting list of department as dict id:[name, short_name]"""
    dept_list = {}
    resp = db_filter_req('state_departments', 'parent_id', ApeksAPI.APEKS_DEPT_ID)
    for dept in resp:
        dept_list[dept['id']] = [dept['name'], dept['name_short']]
    return dept_list


def lessons_staff():
    """Get staff for lesson_id ( {lesson_id:staff_id} )"""
    lessons_staff = {}
    resp = db_request('schedule_day_schedule_lessons_staff')
    for less in resp:
        if less.get('lesson_id') in lessons_staff:
            lessons_staff[less.get('lesson_id')].append(less.get('staff_id'))
        else:
            lessons_staff[less.get('lesson_id')] = [less.get('staff_id')]
    return lessons_staff


def load_subgroups():
    """Get group_id for subgroup_id ( {subgroup_id:group_id} )"""
    subgroups = {}
    resp = db_request('load_subgroups')
    for sub in resp:
        subgroups[sub.get('id')] = sub.get('group_id')
    return subgroups


def load_groups():
    """Get group info by group_id ( {id:[{}] )"""
    groups = {}
    resp = db_request('load_groups')
    for group in resp:
        groups[group.get('id')] = group
    return groups


def plan_education_plans_education_forms():
    """Get education_form_id info by education_plan_id ( {education_plan_id:education_form_id} )"""
    education_forms = {}
    resp = db_request('plan_education_plans_education_forms')
    for plan in resp:
        education_forms[plan.get('education_plan_id')] = plan.get('education_form_id')
    return education_forms


def plan_education_plans():
    """Get plan info by plan_id ( {id:[{}] )"""
    education_plans = {}
    resp = db_request('plan_education_plans')
    for plan in resp:
        education_plans[plan.get('id')] = plan
    return education_plans


def get_lesson_type(lesson):
    if lesson.get("class_type_id") == "1":  # Лекция
        l_type = "lecture"
    elif lesson.get("class_type_id") == "2":  # Семинар
        l_type = "seminar"
    elif (
        lesson.get("class_type_id") == "3"
        or lesson.get("control_type_id") == "12"
        or lesson.get("control_type_id") == "13"
    ):  # П/з, входной, выходной контроль
        l_type = "pract"
    elif lesson.get("control_type_id") == "15":  # Консультации
        l_type = "group_cons"
    elif (
        lesson.get("control_type_id") == "2"
        or lesson.get("control_type_id") == "6"
        or lesson.get("control_type_id") == "10"
    ):  # Зачет, зачет с оценкой, итоговая письменная аудиторная к/р
        l_type = "zachet"
    elif lesson.get("control_type_id") == "1":  # Экзамен
        l_type = "exam"
    elif lesson.get("control_type_id") == "14":  # Итоговая аттестация
        l_type = "final_att"
    else:
        l_type = None
    return l_type


def get_student_type(lesson):
    if lesson.get("education_form_id") == "1" and (
        lesson.get("education_level_id") == "3"
        or lesson.get("education_level_id") == "5"
    ):  # Очно, бакалавр или специалитет
        s_type = "och"
    elif lesson.get("education_form_id") == "3" and (
        lesson.get("education_level_id") == "3"
        or lesson.get("education_level_id") == "5"
    ):  # заочно, бакалавр или специалитет
        s_type = "zo_high"
    elif (
        lesson.get("education_form_id") == "3"
        and lesson.get("education_level_id") == "2"
    ):  # зачно, сренднее
        s_type = "zo_mid"
    elif lesson.get("education_level_id") == "7":  # адъюнктура
        s_type = "adj"
    elif lesson.get("education_form_id") == "7":  # проф подготовка
        s_type = "prof_p"
    elif lesson.get("education_form_id") == "5":  # дополнительное проф образование
        s_type = "dpo"
    else:
        s_type = None
    return s_type


class LoadData:
    def __init__(self, year, month):
        self.year = year
        self.month = month
        self.departments = get_departments()
        self.prepod_dept_structure = self.prepod_dept_structure()
        self.lesson_staff = lessons_staff()
        self.load_subgroups = load_subgroups()
        self.load_groups = load_groups()
        self.plan_education_plans_education_forms = plan_education_plans_education_forms()
        self.plan_education_plans = plan_education_plans()
        self.structured_lessons = self.process_lessons()
        self.control_lessons = self.control_lessons()

    def prepod_dept_structure(self):
        """Department Id for prepod in current month ( {staff_id:department_id} )"""
        structure = {}
        staff = EducationStaff(self.year, self.month)
        for d in self.departments:
            staff_list = staff.staff_list(d)
            for s in staff_list:
                structure[s] = d
        return structure

    def process_lessons(self):
        """Current month lessons list with all data for load report"""
        structured_lessons = []
        for lesson in get_lessons(self.year, self.month):
            if not lesson.get('group_id'):
                if lesson.get('subgroup_id'):
                    lesson['group_id'] = self.load_subgroups.get(lesson['subgroup_id'])
            education_plan_id = self.load_groups.get(lesson.get('group_id'))['education_plan_id']
            education_form_id = self.plan_education_plans_education_forms.get(education_plan_id)
            education_level_id = self.plan_education_plans.get(education_plan_id)['education_level_id']
            lesson['education_plan_id'] = education_plan_id
            lesson['education_form_id'] = education_form_id
            lesson['education_level_id'] = education_level_id
            if lesson.get('id') in self.lesson_staff:
                for prep in self.lesson_staff[lesson['id']]:
                    less_copy = copy(lesson)
                    less_copy['staff_id'] = prep
                    less_copy['department_id'] = self.prepod_dept_structure.get(prep)
                    less_copy['hours'] = 2
                    structured_lessons.append(less_copy)
        return structured_lessons

    def control_lessons(self):
        """Control lessons (1-экз, 2-зач, 6-зач.оц, 14-ИА, 16-канд.экз)"""
        def check_and_process(lesson):
            lookup = ['staff_id', 'discipline_id', 'group_id', 'control_type_id', 'date']
            counter = 0
            for c in control_less:
                if [lesson[val] for val in lookup] == [c[val] for val in lookup]:
                    c['hours'] += 2
                    c['lesson_time_id'] = c.get('lesson_time_id') + ", " + lesson.get('lesson_time_id')
                    counter += 1
            if counter != 0:
                return True
            else:
                return False
        control_less = []
        for l in self.structured_lessons:
            if l.get('control_type_id') in ['1', '2', '6', '14', '16']:
                if not check_and_process(l):
                    control_less.append(copy(l))
        return control_less

    def get_control_hours(self, contr_less):
        """Getting hours for control lessons depending on students number, education type etc..."""
        adj_kf = 1
        zach_kf = 0.25
        exam_kf = 0.3
        final_kf = 0.5
        cont_type = get_lesson_type(contr_less)
        stud_type = get_student_type(contr_less)
        group_id = contr_less.get('group_id')
        people_count = self.load_groups[group_id].get('people_count')
        if stud_type == 'prof_p' or stud_type == 'dpo':
            return contr_less['hours']
        elif stud_type == 'adj' and (contr_less.get('control_type_id') == '14' or contr_less.get('control_type_id') == '16'):
            value = int(people_count) * adj_kf
            return 8 if value > 8 else value
        elif cont_type == 'zachet':
            value = int(people_count) * zach_kf
            return 6 if value > 6 else value
        elif cont_type == 'exam':
            value = int(people_count) * exam_kf
            return 8 if value > 8 else value
        elif cont_type == 'final_att':
            value = int(people_count) * final_kf
            return 8 if value > 8 else value
        # ["och", "zo_high", "zo_mid", "adj", "prof_p", "dpo"]
        # 'zachet', 'exam', 'final_att']

    def unknown_lessons(self):
        """Lessons with inactive staff (which doesn't work in department in selected time)"""
        unknown = []
        for l in self.structured_lessons:
            if not l.get('department_id'):
                unknown.append(l)
        return unknown

    def department_lessons(self, department_id):
        """Select department lessons for load report"""
        dept_lessons = []
        for l in self.structured_lessons:
            if l.get('department_id') == str(department_id):
                dept_lessons.append(l)
        return dept_lessons

class DeptPrepodLoad:
    def __init__(self, staff_list):
        self.load = {}
        self.staff_list = staff_list
        lesson_types = ['lecture', 'seminar', 'pract', 'group_cons', 'zachet', 'exam', 'final_att']
        student_types = ["och", "zo_high", "zo_mid", "adj", "prof_p", "dpo"]
        for staff_id in staff_list:
            self.load[staff_id] = {}
            for l_type in lesson_types:
                self.load[staff_id][l_type] = {}
                for s_type in student_types:
                    self.load[staff_id][l_type][s_type] = 0

    def add_load(self, staff_id, l_type, s_type, value=2):
        self.load[staff_id][l_type][s_type] += value

    def get_load(self, staff_id):
        prep_load = {self.staff_list[staff_id]: self.load[staff_id]}
        return prep_load

class LoadReport:
    def __init__(self, year, month, department_id):
        self.year = year
        self.month = month
        self.department_id = str(department_id)
        self.staff_list = EducationStaff(year, month).staff_list(department_id)
        self.load = LoadData(year, month)
        self.dept_load = DeptPrepodLoad(self.staff_list)
        self.unprocessed = []
        for lesson in self.load.structured_lessons:
            if lesson.get('department_id') == self.department_id:
                staff_id = lesson.get('staff_id')
                l_type = get_lesson_type(lesson)
                s_type = get_student_type(lesson)
                if staff_id and l_type and s_type:
                    if l_type in ['lecture', 'seminar', 'pract', 'group_cons']:
                        self.dept_load.add_load(staff_id, l_type, s_type)
                else:
                    self.unprocessed.append(lesson)
        for control in self.load.control_lessons:
            if control.get('department_id') == self.department_id:
                staff_id = control.get('staff_id')
                l_type = get_lesson_type(control)
                s_type = get_student_type(control)
                if staff_id and l_type and s_type:
                    if l_type in ['zachet', 'exam', 'final_att']:
                        value = load.get_control_hours(control)
                        self.dept_load.add_load(staff_id, l_type, s_type, value)
                else:
                    self.unprocessed.append(control)
        self.data = self.dept_load.load

    def generate_report(self):
        filename = f'{self.year}-{self.month} {self.load.departments.get(self.department_id)[1]}.xlsx'
        wb = load_workbook(FlaskConfig.TEMP_FILE_DIR + "load_report_temp.xlsx")
        ws = wb.active
        ws.title = f'{self.year}-{self.month} {self.load.departments.get(self.department_id)[1]}'
        ws.cell(1, 1).value = self.load.departments.get(self.department_id)[0]
        ws.cell(2, 1).value = f'отчет о нагрузке за {self.month} - {self.year}'
        row = 8
        for prepod in self.data:
            #Style apply
            for i in range(2, 73):
                ws.cell(row, i).style = ExcelStyle.Number
            # Prepod Name
            ws.cell(row, 1).value = self.staff_list[prepod]
            ws.cell(row, 1).style = ExcelStyle.Base
            for l_type in self.data[prepod]:
                if l_type == 'lecture':
                    column = 2
                elif l_type == 'seminar':
                    column = 8
                elif l_type == 'pract':
                    column = 14
                elif l_type == 'group_cons':
                    column = 24
                    if self.data[prepod][l_type]['dpo']:
                        del self.data[prepod][l_type]['dpo']
                elif l_type == 'zachet':
                    column = 29
                elif l_type == 'exam':
                    column = 35
                elif l_type == 'final_att':
                    column = 59
                else:
                    column = 73
                for key, val in self.data[prepod][l_type].items():
                    val="" if val == 0 else val
                    ws.cell(row, column).value = val
                    if val and val % 1 > 0:
                        ws.cell(row, column).number_format = '0.00'
                    column += 1
            ws.cell(row, 72).value = "=SUM(B"+str(row)+":BS"+str(row)+")"
            row += 1
            #Total
            ws.cell(row, 1).value = "Итого"
            ws.cell(row, 1).style = ExcelStyle.BaseBold
            for col in range(2, 73):
                letter = ws.cell(row, col).column_letter
                ws.cell(row, col).value = "=IF(SUM("+letter+"8:"+letter+str(row-1)+")>0,SUM("+letter+"8:"+letter+str(row-1)+"),\"\")"
                ws.cell(row, col).style = ExcelStyle.Number
                ws.cell(row, col).font = Font(name="Times New Roman", size=10, bold=True)
        wb.save(FlaskConfig.EXPORT_FILE_DIR + filename)

In [13]:
load = LoadData(2021, 12)

In [27]:
def get_control_hours(contr_less):
    """Getting hours for control lessons depending on students number, education type etc..."""
    adj_kf = 1
    zach_kf = 0.25
    exam_kf = 0.3
    final_kf = 0.5
    cont_type = get_lesson_type(contr_less)
    stud_type = get_student_type(contr_less)
    group_id = contr_less.get('group_id')
    people_count = load.load_groups[group_id].get('people_count')
    if stud_type == 'prof_p' or stud_type == 'dpo':
        return contr_less['hours']
    elif stud_type == 'adj' and (contr_less.get('control_type_id') == '14' or contr_less.get('control_type_id') == '16'):
        value = int(people_count) * adj_kf
        return 8 if value > 8 else value
    elif cont_type == 'zachet':
        value = int(people_count) * zach_kf
        return 6 if value > 6 else value
    elif cont_type == 'exam':
        value = int(people_count) * exam_kf
        return 8 if value > 8 else value
    elif cont_type == 'final_att':
        value = int(people_count) * final_kf
        return 8 if value > 8 else value


In [107]:
load.structured_lessons

[{'id': '9037',
  'schedule_id': '89',
  'discipline_id': '4',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '1',
  'topic_code': '10.',
  'topic_name': 'Город, в котором получаешь высшее образование.',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '7483',
  'education_plan_id': '50',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '287',
  'department_id': '22',
  'hours': 2},
 {'id': '9049',
  'schedule_id': '89',
  'discipline_id': '36',
  'class_type_id': '1',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '17',
  'topic_name': 'Деятельность органов внутренних дел в борьбе с преступностью',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_cla

In [4]:
d = []
for l in load.control_lessons:
    if l.get('group_id') == '17':
        d.append(l)

In [29]:
d[1]

{'id': '9629',
 'schedule_id': '89',
 'discipline_id': '4',
 'class_type_id': None,
 'control_type_id': '2',
 'date': '2021-12-14',
 'lesson_time_id': '5',
 'topic_code': '',
 'topic_name': '',
 'classroom_id': '102',
 'group_id': '17',
 'subgroup_id': None,
 'flow': None,
 'is_empty': '0',
 'self_work': '0',
 'skip_classroom_check': '0',
 'fixed': '1',
 'journal_lesson_id': '18599',
 'education_plan_id': '50',
 'education_form_id': '1',
 'education_level_id': '5',
 'staff_id': '124',
 'department_id': None,
 'hours': 2}

In [30]:
get_control_hours(d[1])

2.5

In [11]:
a = d[1]
group_id = a.get('group_id')
print(a)
print(load.load_groups[group_id].get('people_count'))
print(load.get_control_hours(a))

{'id': '9629', 'schedule_id': '89', 'discipline_id': '4', 'class_type_id': None, 'control_type_id': '2', 'date': '2021-12-14', 'lesson_time_id': '5', 'topic_code': '', 'topic_name': '', 'classroom_id': '102', 'group_id': '17', 'subgroup_id': None, 'flow': None, 'is_empty': '0', 'self_work': '0', 'skip_classroom_check': '0', 'fixed': '1', 'journal_lesson_id': '18599', 'education_plan_id': '50', 'education_form_id': '1', 'education_level_id': '5', 'staff_id': '124', 'department_id': None, 'hours': 2}
10
2


In [147]:
a = 2
b = 0.25

In [148]:
c = a * b
print(c)

0.5


6

In [59]:
control = []
for l in load.structured_lessons:
    if l.get('control_type_id') in ['1', '2', '6', '14', '16']:
        if l in control:
            print(control.index(l))
            record = control.pop(control.index(l))
            record['hours'] += 2
            control.append(record)
        else:
            control.append(l)

In [55]:
load.structured_lessons

[{'id': '9037',
  'schedule_id': '89',
  'discipline_id': '4',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'topic_code': '10.',
  'topic_name': 'Город, в котором получаешь высшее образование.',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'education_plan_id': '50',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '287',
  'department_id': '22',
  'hours': 2},
 {'id': '9049',
  'schedule_id': '89',
  'discipline_id': '36',
  'class_type_id': '1',
  'control_type_id': None,
  'date': '2021-12-01',
  'topic_code': '17',
  'topic_name': 'Деятельность органов внутренних дел в борьбе с преступностью',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'education_plan_id': '50',
  'education_for

In [60]:
load.control_lessons

[{'id': '14834',
  'schedule_id': '143',
  'discipline_id': '42',
  'class_type_id': None,
  'control_type_id': '2',
  'date': '2021-12-01',
  'topic_code': None,
  'topic_name': None,
  'classroom_id': '53',
  'group_id': '72',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'education_plan_id': '18',
  'education_form_id': '7',
  'education_level_id': None,
  'staff_id': '103',
  'department_id': '21',
  'hours': 2},
 {'id': '19356',
  'schedule_id': '136',
  'discipline_id': '357',
  'class_type_id': None,
  'control_type_id': '2',
  'date': '2021-12-01',
  'topic_code': '',
  'topic_name': '',
  'classroom_id': '60',
  'group_id': '29',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'education_plan_id': '68',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '50',
  'department_id': '17',
  'hours': 2},
 

In [14]:
load.load_groups

{'2': {'id': '2',
  'name': '20з1пд',
  'department_id': '24',
  'education_plan_id': '7',
  'flow': None,
  'people_count': '23',
  'start_date': None,
  'hide_personal': '0',
  'active': '1'},
 '3': {'id': '3',
  'name': '20о5г',
  'department_id': '25',
  'education_plan_id': '3',
  'flow': '1',
  'people_count': '23',
  'start_date': None,
  'hide_personal': '0',
  'active': '1'},
 '4': {'id': '4',
  'name': '20о6г',
  'department_id': '25',
  'education_plan_id': '3',
  'flow': '1',
  'people_count': '22',
  'start_date': None,
  'hide_personal': '0',
  'active': '1'},
 '5': {'id': '5',
  'name': '20о7г',
  'department_id': '25',
  'education_plan_id': '3',
  'flow': '1',
  'people_count': '22',
  'start_date': None,
  'hide_personal': '0',
  'active': '1'},
 '6': {'id': '6',
  'name': '21пк23',
  'department_id': '23',
  'education_plan_id': '28',
  'flow': None,
  'people_count': '9',
  'start_date': '2021-04-27',
  'hide_personal': '0',
  'active': '1'},
 '7': {'id': '7',
  'na

In [109]:
report = LoadReport(2021, 12, 12)

In [110]:
report.generate_report()

In [179]:
report.staff_list

{'30': 'Матросова Л.Д.',
 '32': 'Семенов Е.Ю.',
 '33': 'Шумилин В.П.',
 '54': 'Митряев И.С.'}

In [213]:
for prepod in report.data:
    print(prepod)
    for l_type in report.data[prepod]:
        print(l_type)

30
lecture
seminar
pract
group_cons
zachet
exams
final_att
32
lecture
seminar
pract
group_cons
zachet
exams
final_att
33
lecture
seminar
pract
group_cons
zachet
exams
final_att
54
lecture
seminar
pract
group_cons
zachet
exams
final_att


In [35]:
report.generate_report()

In [142]:
report.load.departments.get('12')[0]

'кафедра информационных технологий в деятельности ОВД'

In [166]:
for key, val in report.data['30']['lecture'].items():
    print(key, val)

och 0
zo_high 0
zo_mid 0
adj 0
prof_p 0
dpo 0


In [87]:
year = 2021
month = 12
department_id = 12

staff_list = EducationStaff(year, month).staff_list(department_id)
load = LoadData(year, month)
report = DeptPrepodLoad(staff_list)

for lesson in load.structured_lessons:
    if lesson.get('department_id') == str(department_id):
        print(lesson.get('department_id'))
        staff_id = lesson.get('staff_id')
        l_type = get_lesson_type(lesson)
        s_type = get_student_type(lesson)
        print(staff_id, l_type, s_type)
        if staff_id and l_type and s_type:
            print(staff_id, l_type, s_type)
            report.add_load(staff_id, l_type, s_type)
        else:
            report.unprocessed.append(lesson)

12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
54 seminar och
54 seminar och
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
33 pract prof_p
33 pract prof_p
12
33 pract prof_p
33 pract prof_p
12
32 pract och
32 pract och
12
32 pract och
32 pract och
12
54 pract och
54 pract och
12
54 pract och
54 pract och
12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
33 pract och
33 pract och
12
33 pract och
33 pract och
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
33 pract prof_p
33 pract prof_p
12
33 zachet och
33 zachet och
12
33 zachet och
33 zachet och
12
54 zachet zo_high
54 

In [88]:
report.load

{'30': {'lecture': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'seminar': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'pract': {'och': 8,
   'zo_high': 20,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 8},
  'group_cons': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'zachet': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'exams': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'final_att': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0}},
 '32': {'lecture': {'och': 0,
   'zo_high': 2,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'seminar': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'pract': {'och': 6,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
  

In [80]:
load.structured_lessons

[{'id': '9037',
  'schedule_id': '89',
  'discipline_id': '4',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '1',
  'topic_code': '10.',
  'topic_name': 'Город, в котором получаешь высшее образование.',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '7483',
  'education_plan_id': '50',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '287',
  'department_id': '22'},
 {'id': '9049',
  'schedule_id': '89',
  'discipline_id': '36',
  'class_type_id': '1',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '17',
  'topic_name': 'Деятельность органов внутренних дел в борьбе с преступностью',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check':

In [33]:
load = LoadData(2021, 12)

In [127]:
a = {'c': 2}
b = a

In [129]:
b['c'] = 3

In [28]:
a = 4.52

In [29]:
a % 1

0.5199999999999996

In [7]:
row = 11

In [8]:
str(row-8)

'3'

In [27]:
wb = load_workbook(FlaskConfig.TEMP_FILE_DIR + "load_report_temp.xlsx")
ws = wb.active

In [32]:
ws.cell(2, 1).column_letter

'A2'